
Trong notebook này, chúng ta sẽ thử quy trình lập trình RNN với Keras để phân loại các câu văn bản.

I.   **Trước tiên**, chúng ta sẽ nhập các thư viện hữu ích.

II.   **Sau đó**, chúng ta sẽ load dữ liệu và tạo ma trận word embedding bằng Glove.

III.  **Chúng ta sẽ thử một mô hình RNN đơn giản** rồi đánh giá chất lượng của nó.

IV. Cuối cùng, chúng ta sẽ sử dụng các kỹ thuật để gia tăng độ chính xác của mô hình.

**Task 1:** Thiết lập Fre GPU trong notebook này.

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-f0f54757-3dc9-16c4-0dfe-9e01c3648620)


## Gắn Google Drive cục bộ
**Task 2:** Gắn Google vào Google Colab Driver.


In [1]:
## VIẾT CODE cho task 2 ở đây:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# I. Nhập tất cả các thư viện hữu ích.

In [4]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import tensorflow.keras
import datetime
from tensorflow.keras import backend as K
import tensorflow.keras.optimizers as Optimizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, load_model
import tensorflow_addons as tfa

from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from sklearn.metrics import confusion_matrix as CM
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
import matplotlib.pyplot as plot
import seaborn as sn

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
pip install tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 9.0 MB/s eta 0:00:00


**Task 3**: Copy tập dữ liệu từ Google Drive vào Colab

In [5]:
## VIẾT CODE cho task 3 ở đây:
data_dir = '/content/drive/MyDrive/FUNIX/MON4/ASM2/'

# II. Load dữ liệu.

## Về tập dữ liệu.
Câu hỏi không hợp lệ là những câu hỏi nhằm đưa ra một tuyên bố thay vì tìm kiếm những câu trả lời hữu ích. Một số đặc điểm cho thấy câu hỏi không hợp lệ gồm:

* Có giọng điệu không trung lập.
* Có tính chê bai hoặc kích động.
* Không có căn cứ thực tế.
* Sử dụng nội dung khiêu dâm (loạn luân, thú tính, ấu dâm) để gây sốc, không phải để tìm kiếm câu trả lời xác thực.

Dữ liệu bao gồm câu hỏi đã đề ra và liệu nó có được xác định là không hợp lệ hay không (target = 1).

**Task 4**: Load tập dữ liệu.
* Load dữ liệu từ file CSV.
* Xóa tất cả các hàng có giá trị NA.
* Chia dữ liệu thành 3 tập: Tập huấn luyện, tập kiểm định và tập kiểm tra (0.9/0.05/0.05, random_seed = 9) với cùng một tỷ lệ số dữ liệu giữa mỗi lớp.
* In ra mô tả của tập dữ liệu này.




In [6]:
def load_data(data_link):
    '''
    input: data link.
    output:
        train_set, validation_set và test_set(0.95/0.05/0.05) mà không có các giá trị NA.
    '''
    ## VIẾT CODE cho task 4 ở đây:
    file_name = data_dir + data_link
    data = pd.read_csv(file_name)
    data.rename(columns={'target':'label'}, inplace=True)
    data = data.dropna()
    train, validation, test = np.split(data.sample(frac=1),[int(0.9*len(data)), int(0.95*len(data))])

    return train, validation, test
np.random.seed(9)
train_set, validation_set, test_set = load_data('train.csv')
print(train_set['label'].describe())
print(validation_set['label'].describe())
print(test_set['label'].describe())

count    1.175509e+06
mean     6.190340e-02
std      2.409801e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
Name: label, dtype: float64
count    65306.000000
mean         0.062261
std          0.241630
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: label, dtype: float64
count    65307.000000
mean         0.060882
std          0.239115
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: label, dtype: float64


# Mã hóa dữ liệu văn bản.
Hãy khai báo một số tham số cơ bản trước:

In [7]:
embed_size = 50 # mỗi vectơ từ lớn bao nhiêu
max_features = 20000 # cần sử dụng bao nhiêu từ duy nhất (tức là số hàng trong vectơ embedding)
max_len = 50 # sử dụng số từ tối đa trong câu hỏi


**Task 5:** Mã hóa tập dữ liệu bằng Tokenizer và vectơ biểu diễn one-hot.
* Mã hóa văn bản (cột question_text) bằng cách chuyển từng question text thành danh sách chỉ mục từ bằng [Tokenizer](https://stackoverflow.com/questions/51956000/what-does-keras-tokenizer-method-exactly-do) với **max_features** và tất cả các câu văn bản từ tập huấn luyện và tập kiểm định.
* Chuyển từng danh sách chỉ mục từ thành độ dài như nhau - **max_len** (có cắt tỉa hoặc đệm nếu cần) bằng cách sử dụng [pad_sequences](https://keras.io/preprocessing/sequence/).
* Mã hóa nhãn (cột nhãn) bằng cách sử dụng hàm [to_categorical](https://keras.io/utils/) trong Keras.

In [21]:
train_sentences = train_set['question_text']
validation_sentences = validation_set['question_text']
test_sentences = test_set['question_text']
train_labels = train_set['label']
validation_labels = validation_set['label']
test_labels = test_set['label']

In [22]:
def encoding_textdata(train_set, validation_set, test_set, max_features, max_len):
    '''
    Input:
    - Train/validation/test dataset.
    - max_features, max_len.
    Output:
    - X train/validation/test, y train/validation/test.
    - Tokenizer.
    '''
    ## VIẾT CODE cho task 5 ở đây:

    tokenizer = Tokenizer(num_words = max_features)
    tokenizer.fit_on_texts(list(train_sentences))

    X_tr = tokenizer.texts_to_sequences(train_sentences)
    X_va = tokenizer.texts_to_sequences(validation_sentences)
    X_te = tokenizer.texts_to_sequences(test_sentences)

    X_tr = pad_sequences(X_tr, maxlen=max_len)
    X_va = pad_sequences(X_va, maxlen=max_len)
    X_te = pad_sequences(X_te, maxlen=max_len)

    y_tr = to_categorical(train_labels)
    y_va = to_categorical(validation_labels)
    y_te = to_categorical(test_labels)

    return (X_tr, y_tr), (X_va, y_va), (X_te, y_te), tokenizer

(X_tr, y_tr), (X_va, y_va), (X_te, y_te), tokenizer = encoding_textdata(train_set, validation_set, test_set, max_features, max_len)

**Task 6**: Tạo ma trận word embedding.
* Đầu tiên, hãy viết một hàm để load [GloVe dictionary.](https://medium.com/analytics-vidhya/basics-of-using-pre-trained-glove-vectors-in-python-d38905f356db)
* Sau đó, tạo ma trận word embedding bằng GloVe dictionary với các tham số sau:
    - Shape của ma trận word embedding: (Number of word, embed_size).
    - Embed size: 50.
    - Number of words: Tối thiểu của (max_features, len(word_index)), trong khi word_index là dictionary của từ chứa trong tokenizer.
    - Nếu một từ xuất hiện trong GloVe dictionary, chúng ta nên lấy giá trị khởi tạo của nó như trong GloVe dictionary. Nếu không, hãy lấy một giá trị ngẫu nhiên bình thường với mean và std làm mean và std của giá trị GloVe dictionary.
    



In [24]:

def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')
def get_GloVe_dict(GloVe_link):
    '''
    input: GloVe link.
    output: GloVe dictionary.
    '''
    ## VIẾT CODE cho task 6 ở đây:
    link = '/content/drive/MyDrive/FUNIX/MON4/ASM2/' + GloVe_link
    embeddings_dict = {}
    with open(link, 'r', encoding='utf-8') as f:
      for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], 'float32')
        embeddings_dict[word] = vector
    return embeddings_dict
GloVe_link = 'glove.6B.50d.txt'
GloVe_dict = get_GloVe_dict(GloVe_link)

In [25]:
def create_embedding_matrix(GloVe_dict, tokenizer, max_features):
    '''
    input: GloVe dictionaray, tokenizer từ tập huấn luyện và tập kiểm định, số lượng đặc trưng tối đa.
    output: Word embedding matrix.
    '''

    ## VIẾT CODE cho task 6 ở đây:
    all_embs = np.stack(list(GloVe_dict.values()))
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    number_of_words = min(max_features, (len(tokenizer.word_index)+1))
    #number_of_words = len(tokenizer.word_index)
    embedding_matrix = np.random.normal(emb_mean, emb_std, (number_of_words, embed_size))
    for word, i in tokenizer.word_index.items():
      i -= 1
      embedding_value = GloVe_dict.get(word)
      if embedding_value is not None and i < number_of_words:
        embedding_matrix[i] = embedding_value

    return embedding_matrix

embedding_matrix = create_embedding_matrix(GloVe_dict, tokenizer, max_features)

# III. Lập mô hình
Chúng ta cần hoàn thành một số bước:

Xây dựng mô hình.

Biên dịch mô hình.

Huấn luyện/khớp dữ liệu với mô hình.

Đánh giá mô hình trên tập kiểm tra.

## Xây dựng mô hình
**Task 7:** Chúng ta có thể xây dựng một mô hình dễ dàng gồm các layer khác nhau như:
* Layer [Embedding](https://keras.io/layers/embeddings/) với max_features, embed_size và embedding_matrix.
* [Bidirectional LSTM layer](https://keras.io/examples/nlp/bidirectional_lstm_imdb/?fbclid=IwAR3fEd6aWyeIDEhZSspjtCRiP0c0Jnz5-XdnUHQYwX8Tp8k9Ni4I8Q5tP9o) với số lượng trạng thái ẩn = 50, dropout_rate = 0.1 và recurrent_dropout_rate = 0.1.
* GlobalMaxPool1D.
* Dense với số nút = 50, activation = 'relu'.
* Dropout với rate = 0.1.
* Final dense với số nút = số class, activation = 'sigmoid'.

In [26]:
def create_model(max_len, max_features, embed_size):
    '''
    input: max_len, max_features, embed_size
    output: model.
    '''
    ## VIẾT CODE cho task 7 ở đây:
    inputs = Input(shape=(max_len, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inputs)
    x = Bidirectional(LSTM(50,return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
    x = GlobalMaxPool1D()(x)
    x = Dense(50, activation='relu')(x)
    x = Dropout(0.1)(x)
    outputs = Dense(2, activation='sigmoid')(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model

model = create_model(max_len, max_features, embed_size)

**Task 8:** Biên dịch mô hình và thiết lập callback. Sau đó in ra model summary.
* [Biên dịch](https://keras.io/models/model/#compile) mô hình với Adam Optimizer, lr = 1e-2, loss phù hợp cho bài toán phân loại nhị phân và ["F1-score"](https://github.com/tensorflow/addons/issues/825) là phép đo.
* In ra model summary.

In [27]:
def optimize(model):
    '''
    Input:
        Mô hình.
    Return:
        Mô hình đã biên dịch.
    '''
    ## VIẾT CODE cho task 8 ở đây:
    model.compile(loss = 'binary_crossentropy',
                  optimizer = optimizers.Adam(learning_rate=1e-2),
                  metrics = ['F1-score'])
    return model

model = optimize(model)
print(model.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 50)]              0         
                                                                 
 embedding_3 (Embedding)     (None, 50, 50)            1000000   
                                                                 
 bidirectional_3 (Bidirecti  (None, 50, 100)           40400     
 onal)                                                           
                                                                 
 global_max_pooling1d_1 (Gl  (None, 100)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_2 (Dense)             (None, 50)                5050      
                                                                 
 dropout_1 (Dropout)         (None, 50)                0   

**Task 9**: Thiết lập callback.
* Tạo [tensorboard callback](https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks) để lưu logs.
* Tạo [checkpoint callback](https://machinelearningmastery.com/check-point-deep-learning-models-keras/) để lưu checkpoint với độ chính xác tốt nhất sau mỗi epoch.
* Tạo [ReduceLROnPlateau](https://keras.io/callbacks/#reducelronplateau) callback với factor=0.3, patience=1 và "Validation F1-score" monitor.
* Tạo [early stopping callback](https://keras.io/callbacks/#earlystopping) với patience=7, mode = 'max' và "Validation F1-score" monitor.



In [28]:
def callback_model(checkpoint_name, logs_name):
    '''
    Input:
        Checkpoint name, logs name tốt nhất.
    Return:
        Callback list có chứa tensorboard callback và checkpoint callback.
    '''
    ## VIẾT CODE cho task 9 ở đây:
    logdir = os.path.join(logs_name, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    callbacks_list = [TensorBoard(log_dir=logdir),
                      ModelCheckpoint(filepath=checkpoint_name, monitor='val_accuracy', save_best_only=True, mode='max'),
                      tensorflow.keras.callbacks.ReduceLROnPlateau(factor=0.3, patience=1, monitor='Validation F1-score'),
                      tensorflow.keras.callbacks.EarlyStopping(patience=7, mode='max', monitor='Validation F1-score')
                      ]

    return callbacks_list

checkpoint_name = 'weights.best.hdf5'
logs_name = 'training_logs'
callbacks_list = callback_model(checkpoint_name, logs_name)

In [29]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 50)]              0         
                                                                 
 embedding_3 (Embedding)     (None, 50, 50)            1000000   
                                                                 
 bidirectional_3 (Bidirecti  (None, 50, 100)           40400     
 onal)                                                           
                                                                 
 global_max_pooling1d_1 (Gl  (None, 100)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_2 (Dense)             (None, 50)                5050      
                                                                 
 dropout_1 (Dropout)         (None, 50)                0   

**Task 10:** Huấn luyện mô hình.

* Huấn luyện mô hình với 20 epoch với batch_size = 4096.
* Trả về mô hình có trọng số checkpoint tốt nhất.

*Gợi ý*: Trước tiên hãy khớp mô hình, sau đó reload mô hình (hàm load_model) với trọng số checkpoint tốt nhất.

In [44]:
def train_model(model, callbacks_list):
    '''
    Input:
        Mô hình và callback list,
    Return:
        Mô hình với trọng số checkpoint tốt nhất.
    '''
    ## VIẾT CODE cho task 10 ở đây:
    epochs = 20
    batch_size = 4096
    model_1 = model.fit(X_tr, y_tr,
                        batch_size = batch_size,
                        epochs = epochs,
                        validation_data = (X_va, y_va),
                        callbacks = callbacks_list)
    #model = load_model('weights.best.hdf5')

    return model

model = train_model(model, callbacks_list)


Epoch 1/20


TypeError: ignored

**Task 11:** Hiển thị tensorboard trong notebook.

In [ ]:
## VIẾT CODE cho task 11 ở đây:

**Task 12:** Dự đoán trên tập kiểm tra.

* Hoàn thành hàm get_prediction_classes.
* In ra precision, recall và F1 score.

In [ ]:
def get_prediction_classes(model, X, y):
    ## VIẾT CODE cho task 12 ở đây:
    '''
    Input:
        Mô hình và tập dữ liệu dự đoán.
    Return:
        Prediction list và groundtrurth list với predicted classes.
    '''



    return predictions, groundtruths


test_predictions, test_groundtruths = get_prediction_classes(model,  X_te, y_te)
print(precision_score(test_predictions, test_groundtruths))
print(recall_score(test_predictions, test_groundtruths))
print(f1_score(test_predictions, test_groundtruths))

**Task 13:** Thực hiện kết quả dự đoán trên tập kiểm tra bằng cách sử dụng ma trận nhầm lẫn. Hãy nhớ hiển thị tên lớp trong ma trận nhầm lẫn.

In [ ]:
def plot_confusion_matrix(predictions, groundtruth, class_names):
    ## VIẾT CODE cho task 13 ở đây:




    plot.show()
class_names = ['valid', 'invalid']
plot_confusion_matrix(test_predictions, test_groundtruths, class_names)

**Task 14**: Tinh chỉnh mô hình - tinh chỉnh mô hình bằng cách sử dụng một số phương pháp sau:
* Tăng tối đa epoch, thay đổi batch size.
* Thay thế LSTM bằng các nút GRU và kiểm tra xem nó có thay đổi gì không.
* Thêm một layer LSTM/GRU khác hoặc thay thế nó bằng mô-đun Attention/Transformers, xem có cải thiện gì không.
* Thử với Dense layer (add/# units/...).
* Tìm các quy tắc tiền xử lý mà bạn có thể thêm để cải thiện chất lượng dữ liệu.
* Find another GloVe dictionary. Tìm một GloVe dictionary khác.
Yêu cầu: F1 score phải tăng thêm 2-3%.

In [ ]:
## VIẾT CODE cho task 14 ở đây: